In [11]:
import pandas as pd
import scipy.stats 

In [2]:
bike_path = "clean/bike_data.csv"
city_path = "clean/Cities.csv"
bike_scores = pd.read_csv(bike_path)
weather_scores = pd.read_csv(city_path)
bike_scores

,City,Overall Score,Ridership,Safety,Acceleration,Network,Reach
0,"SPRINGDALE, AR",3.0,1.3,2.2,3.0,2.2,4.0
1,"ENID, OK",2.4,1.0,1.6,NaN,3.2,3.8
2,"DUBUQUE, IA",2.1,1.6,2.3,0.5,2.2,3.8
3,"SOUTH BEND, IN",2.0,1.7,1.9,0.6,2.1,3.7
4,"CRESTED BUTTE, CO",2.6,3.5,2.8,0.7,2.5,3.6
...,...,...,...,...,...,...,...
505,"BRIELLE, NJ",0.6,1.0,0.4,NaN,0.8,0.0
506,"TATAMY, PA",0.5,0.9,0.2,NaN,0.8,0.0
507,"FOXBOROUGH, MA",0.5,0.9,0.2,NaN,0.8,0.0
508,"HOOKSETT, NH",0.5,0.8,0.2,NaN,0.8,0.0


In [3]:
weather_scores = weather_scores.drop('Unnamed: 0', axis = 1)
weather_scores['City'] = weather_scores['City'].str.upper()
weather_scores

,City,Value
0,"ARDEN-ARCADE, CA",1
1,"CITRUS HEIGHTS, CA",1
2,"ROSEVILLE, CA",1
3,"DAVIS, CA",1
4,"GLENDALE, CA",1
...,...,...
595,"SYRACUSE, NY",100
596,"YOUNGSTOWN, OH",100
597,"KETCHIKAN, AK",100
598,"SITKA, AK",100


In [4]:
#one csv had 510 other had 600 left with 317
merge_data = pd.merge(bike_scores, weather_scores, on = 'City')
merge_data

,City,Overall Score,Ridership,Safety,Acceleration,Network,Reach,Value
0,"SPRINGDALE, AR",3.0,1.3,2.2,3.0,2.2,4.0,55
1,"SOUTH BEND, IN",2.0,1.7,1.9,0.6,2.1,3.7,86
2,"KALAMAZOO, MI",1.9,1.7,1.7,0.6,2.2,3.6,85
3,"SPRINGFIELD, MO",1.9,1.8,1.3,0.5,2.2,3.5,63
4,"MEDFORD, OR",1.9,1.1,1.8,NaN,1.6,3.2,39
...,...,...,...,...,...,...,...,...
312,"ROCKFORD, IL",1.3,1.6,1.4,0.5,2.1,0.9,96
313,"CLARKSVILLE, TN",1.1,1.3,1.1,0.3,2.0,0.8,53
314,"CHEYENNE, WY",1.2,1.6,1.3,0.4,2.1,0.7,61
315,"WHEELING, WV",1.2,0.8,1.8,NaN,1.6,0.5,82


In [23]:
merge_data.to_csv('clean/merged_data.csv', index = False, header = True)

In [ ]:
connection_string = "postgres:Chasedog10:5432/best_biking"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
engine.table_names()

In [ ]:
merge_data = merge_data.set_index("City", inplace = True)
weather_scores = weather_scores.set_index("City", inplace = True)


merge_data.to_sql(name='Bike & Weather Scores', con=engine, if_exists='append', index=True)
weather_scores.to_sql(name='Good Weather Score', con=engine, if_exists='append', index=True)

In [14]:
merge_data[['City Name','State']] = merge_data.City.str.split(", ",expand=True) 
#merge_data.to_csv('clean/merege2_data.csv', index = False, header = True)
pregroup = merge_data.drop('City', axis = 1)
pregroup.to_csv('clean/merge2_data.csv', index = False, header = True)
pregroup.head()

,Overall Score,Ridership,Safety,Acceleration,Network,Reach,Value,City Name,State
0,3.0,1.3,2.2,3.0,2.2,4.0,55,SPRINGDALE,AR
1,2.0,1.7,1.9,0.6,2.1,3.7,86,SOUTH BEND,IN
2,1.9,1.7,1.7,0.6,2.2,3.6,85,KALAMAZOO,MI
3,1.9,1.8,1.3,0.5,2.2,3.5,63,SPRINGFIELD,MO
4,1.9,1.1,1.8,NaN,1.6,3.2,39,MEDFORD,OR


In [18]:
state_data = pregroup.groupby(['State']).mean()
state_data = state_data.rename(columns={"Overall Score": "Bike Score", "Value": "Good Weather %ile", "City Name" : "City"})
state_data = state_data[['Good Weather %ile', 'Bike Score', 'Ridership', 'Safety', 'Acceleration', 'Network', 'Reach']]
state_data

,Good Weather %ile,Bike Score,Ridership,Safety,Acceleration,Network,Reach
State,,,,,,,
AK,89.333333,1.866667,1.900000,2.066667,0.533333,2.200000,2.633333
AL,49.833333,1.633333,1.233333,1.383333,1.500000,1.466667,2.133333
AR,56.600000,2.120000,1.360000,2.000000,1.860000,2.000000,2.400000
AZ,15.454545,1.627273,1.645455,1.718182,0.622222,1.909091,1.872727
CA,22.323529,1.832353,1.676471,1.973529,1.133333,1.844118,2.026471
CO,37.125000,2.362500,2.137500,2.325000,2.085714,2.112500,2.037500
CT,76.000000,1.366667,1.283333,1.783333,0.560000,1.116667,2.016667
DC,93.000000,3.100000,3.500000,2.900000,2.400000,2.200000,2.000000
DE,69.500000,1.450000,1.375000,1.800000,0.500000,1.775000,1.725000


In [19]:
state_data.to_csv('clean/by_state.csv', index = False, header = True)